In [1]:
import pandas as pd
import numpy as np
path = r"C:\Users\kpras\Desktop\ClientsCalculatedTAT_13-01-2026 23_22_06_Tata_Motors_Finance_Limited_All.xlsx"
df = pd.read_excel(path)
print(df.head())

   SrNo                       Client                   Product ApplicationNo  \
0     1  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU1  R26925922165   
1     2  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU1  R26925922165   
2     3  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU1  R26925922165   
3     4  Tata Motors Finance Limited  Tata Motors_CV_MHCV_FTU1  R27925924803   
4     5  Tata Motors Finance Limited   Tata Motors_CV_SCV_FTU2  R28025925378   

  Application Status         Case In Time      DE Completed At  \
0            Sampled  Dec 19 2025  3:34PM  Dec 19 2025  3:29PM   
1            Sampled  Dec 19 2025  3:34PM  Dec 17 2025  5:14PM   
2            Sampled  Dec 19 2025  3:34PM  Sep 27 2025  5:01PM   
3           Screened  Dec  5 2025  3:03PM  Oct  9 2025 12:23PM   
4            Sampled  Dec  4 2025 12:57PM  Nov 28 2025 11:33AM   

                DE Completed By        Case Out Time           Processed By  \
0           Abhijeet Nachvankar  Dec 19 202

In [2]:
df_new = df[['Product', 'Application Status']].copy()
df_new['cnt'] = 1

product_sampling = pd.pivot_table(
    df_new,
    index='Product',
    columns='Application Status',
    values='cnt',
    aggfunc='sum',
    fill_value=0,
    margins=True,              # adds Grand Total
    margins_name='Grand Total'
)

print(product_sampling)


Application Status         Sampled  Screened  Grand Total
Product                                                  
Tata Motors_CV_ICV_FTU1         70        42          112
Tata Motors_CV_ICV_FTU2         48        71          119
Tata Motors_CV_ICV_RTLC1         0         5            5
Tata Motors_CV_ICV_SRT1         43        43           86
Tata Motors_CV_ICV_SRT2         14        13           27
Tata Motors_CV_LCV_FTU1         57        67          124
Tata Motors_CV_LCV_FTU2         21        32           53
Tata Motors_CV_LCV_RTLC1         3         4            7
Tata Motors_CV_LCV_RTLC2         7         7           14
Tata Motors_CV_LCV_SRT1         30        38           68
Tata Motors_CV_LCV_SRT2          4        11           15
Tata Motors_CV_LCV_SRT3          4         2            6
Tata Motors_CV_MHCV_FTU1       202       132          334
Tata Motors_CV_MHCV_FTU2        81       111          192
Tata Motors_CV_MHCV_RTLC1       17        30           47
Tata Motors_CV

In [3]:
# df_new = df[['Product', 'Application Status']]

# product_sampling = (
#     df_new.groupby(['Product', 'Application Status'])
#           .size()
#           .unstack(fill_value=0)
# )

# product_sampling['Total'] = product_sampling.sum(axis=1)
# product_sampling.loc['Total'] = product_sampling.sum()


# print(product_sampling)


In [4]:
# keep pivot with margins
product_sampling2 = pd.pivot_table(
    df_new,
    index='Product',
    columns='Application Status',
    values='cnt',
    aggfunc='sum',
    fill_value=0,
    margins=True,
    margins_name='Grand Total'
)

# calculate percentage WITHOUT Grand Total
product_sampling_pct = product_sampling2.div(
    product_sampling.drop(columns='Grand Total').sum(axis=1),
    axis=0
).mul(100).round(2)

print(product_sampling_pct)



Application Status         Sampled  Screened  Grand Total
Product                                                  
Tata Motors_CV_ICV_FTU1      62.50     37.50        100.0
Tata Motors_CV_ICV_FTU2      40.34     59.66        100.0
Tata Motors_CV_ICV_RTLC1      0.00    100.00        100.0
Tata Motors_CV_ICV_SRT1      50.00     50.00        100.0
Tata Motors_CV_ICV_SRT2      51.85     48.15        100.0
Tata Motors_CV_LCV_FTU1      45.97     54.03        100.0
Tata Motors_CV_LCV_FTU2      39.62     60.38        100.0
Tata Motors_CV_LCV_RTLC1     42.86     57.14        100.0
Tata Motors_CV_LCV_RTLC2     50.00     50.00        100.0
Tata Motors_CV_LCV_SRT1      44.12     55.88        100.0
Tata Motors_CV_LCV_SRT2      26.67     73.33        100.0
Tata Motors_CV_LCV_SRT3      66.67     33.33        100.0
Tata Motors_CV_MHCV_FTU1     60.48     39.52        100.0
Tata Motors_CV_MHCV_FTU2     42.19     57.81        100.0
Tata Motors_CV_MHCV_RTLC1    36.17     63.83        100.0
Tata Motors_CV

In [5]:
final_df = product_sampling.join(
    product_sampling_pct,
    how='left',
    lsuffix='_cnt',
    rsuffix='_pct'
)

print(final_df)



Application Status         Sampled_cnt  Screened_cnt  Grand Total_cnt  \
Product                                                                 
Tata Motors_CV_ICV_FTU1             70            42              112   
Tata Motors_CV_ICV_FTU2             48            71              119   
Tata Motors_CV_ICV_RTLC1             0             5                5   
Tata Motors_CV_ICV_SRT1             43            43               86   
Tata Motors_CV_ICV_SRT2             14            13               27   
Tata Motors_CV_LCV_FTU1             57            67              124   
Tata Motors_CV_LCV_FTU2             21            32               53   
Tata Motors_CV_LCV_RTLC1             3             4                7   
Tata Motors_CV_LCV_RTLC2             7             7               14   
Tata Motors_CV_LCV_SRT1             30            38               68   
Tata Motors_CV_LCV_SRT2              4            11               15   
Tata Motors_CV_LCV_SRT3              4             

In [6]:
df['tat_min'] = (
    pd.to_timedelta(df['CalculatedTAT']).dt.total_seconds() / 60
)

In [7]:
df['tat_bucket'] = pd.cut(
    df['CalculatedTAT'],
    bins=[-1, 15, 30, float('inf')],
    labels=['<15 Min', '<30 Min', '>=30 Min']
)


In [8]:
tat_pivot = pd.pivot_table(
    df,
    index='Product',
    columns='tat_bucket',
    values='tat_min',
    aggfunc='count',
    fill_value=0,
    margins=True,
    margins_name='Grand Total'
)

print(tat_pivot)


tat_bucket                 <15 Min  <30 Min  >=30 Min  Grand Total
Product                                                           
Tata Motors_CV_ICV_FTU1         60        1        51          112
Tata Motors_CV_ICV_FTU2         74        0        45          119
Tata Motors_CV_ICV_RTLC1         4        0         1            5
Tata Motors_CV_ICV_SRT1         58        0        28           86
Tata Motors_CV_ICV_SRT2         21        0         6           27
Tata Motors_CV_LCV_FTU1         67        3        54          124
Tata Motors_CV_LCV_FTU2         20        5        28           53
Tata Motors_CV_LCV_RTLC1         6        0         1            7
Tata Motors_CV_LCV_RTLC2        10        0         4           14
Tata Motors_CV_LCV_SRT1         45        0        23           68
Tata Motors_CV_LCV_SRT2          8        1         6           15
Tata Motors_CV_LCV_SRT3          6        0         0            6
Tata Motors_CV_MHCV_FTU1       174       19       141         

C:\Users\kpras\AppData\Local\Temp\ipykernel_17924\2819589356.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  tat_pivot = pd.pivot_table(


In [9]:
tat_pct = tat_pivot.div(
    tat_pivot.drop(columns='Grand Total').sum(axis=1),
    axis=0
).mul(100).round(2)

print(tat_pct)


tat_bucket                 <15 Min  <30 Min  >=30 Min  Grand Total
Product                                                           
Tata Motors_CV_ICV_FTU1      53.57     0.89     45.54        100.0
Tata Motors_CV_ICV_FTU2      62.18     0.00     37.82        100.0
Tata Motors_CV_ICV_RTLC1     80.00     0.00     20.00        100.0
Tata Motors_CV_ICV_SRT1      67.44     0.00     32.56        100.0
Tata Motors_CV_ICV_SRT2      77.78     0.00     22.22        100.0
Tata Motors_CV_LCV_FTU1      54.03     2.42     43.55        100.0
Tata Motors_CV_LCV_FTU2      37.74     9.43     52.83        100.0
Tata Motors_CV_LCV_RTLC1     85.71     0.00     14.29        100.0
Tata Motors_CV_LCV_RTLC2     71.43     0.00     28.57        100.0
Tata Motors_CV_LCV_SRT1      66.18     0.00     33.82        100.0
Tata Motors_CV_LCV_SRT2      53.33     6.67     40.00        100.0
Tata Motors_CV_LCV_SRT3     100.00     0.00      0.00        100.0
Tata Motors_CV_MHCV_FTU1     52.10     5.69     42.22        1

In [10]:
tat_pct = tat_pct.add_suffix(' (%)')

final_tat = tat_pivot.join(tat_pct)
print(final_tat)


tat_bucket                 <15 Min  <30 Min  >=30 Min  Grand Total  \
Product                                                              
Tata Motors_CV_ICV_FTU1         60        1        51          112   
Tata Motors_CV_ICV_FTU2         74        0        45          119   
Tata Motors_CV_ICV_RTLC1         4        0         1            5   
Tata Motors_CV_ICV_SRT1         58        0        28           86   
Tata Motors_CV_ICV_SRT2         21        0         6           27   
Tata Motors_CV_LCV_FTU1         67        3        54          124   
Tata Motors_CV_LCV_FTU2         20        5        28           53   
Tata Motors_CV_LCV_RTLC1         6        0         1            7   
Tata Motors_CV_LCV_RTLC2        10        0         4           14   
Tata Motors_CV_LCV_SRT1         45        0        23           68   
Tata Motors_CV_LCV_SRT2          8        1         6           15   
Tata Motors_CV_LCV_SRT3          6        0         0            6   
Tata Motors_CV_MHCV_

In [17]:
final_df = (
    product_sampling.add_prefix('Status_')
        .join(product_sampling_pct.add_prefix('Status_').add_suffix(' (%)'))
        .join(tat_pivot.add_prefix('TAT_'))
        .join(tat_pct.add_prefix('TAT_').add_suffix(' (%)'))
)
print(final_df)

                           Status_Sampled  Status_Screened  \
Product                                                      
Tata Motors_CV_ICV_FTU1                70               42   
Tata Motors_CV_ICV_FTU2                48               71   
Tata Motors_CV_ICV_RTLC1                0                5   
Tata Motors_CV_ICV_SRT1                43               43   
Tata Motors_CV_ICV_SRT2                14               13   
Tata Motors_CV_LCV_FTU1                57               67   
Tata Motors_CV_LCV_FTU2                21               32   
Tata Motors_CV_LCV_RTLC1                3                4   
Tata Motors_CV_LCV_RTLC2                7                7   
Tata Motors_CV_LCV_SRT1                30               38   
Tata Motors_CV_LCV_SRT2                 4               11   
Tata Motors_CV_LCV_SRT3                 4                2   
Tata Motors_CV_MHCV_FTU1              202              132   
Tata Motors_CV_MHCV_FTU2               81              111   
Tata Mot